Instalamos la librería undetected-chromedriver, que funciona como Selenium pero añade medidas para evitar el ban al hacer WebScraping.

In [2]:
# pip install undetected-chromedriver

Versión que funciona bastante bien. Solo para vehículos de 1 página (65):

In [ ]:
import time
from datetime import datetime
import random
import sqlite3
import os
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup as bs
from include.utils.funciones import *

# Conexión a la BBDD:
con = sqlite3.connect("./include/db_vehiculos.db")
cursor = con.cursor()

# User agents aleatorios:
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:99.0) Gecko/20100101 Firefox/99.0",
]

selected_ua = random.choice(USER_AGENTS)
options = uc.ChromeOptions()
options.add_argument(f"user-agent={selected_ua}")
options.add_argument("--no-sandbox")
options.add_argument("--disable-blink-features=AutomationControlled")
# options.add_argument("--headless")  # opcional


# Iniciar navegador
driver = uc.Chrome(options=options)
driver.get("https://www.coches.net/segunda-mano/?MakeIds%5B0%5D=7&ModelIds%5B0%5D=0")
time.sleep(random.uniform(3, 5))

# Aceptar cookies si aparecen
try:
    Cookies = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div/div/div[3]/button[3]")
    Cookies.click()
    time.sleep(1)
except:
    pass

# Lista de IDs en la BBDD
cursor.execute("SELECT PK_ANUNCIO_ID FROM TX_VEHICULOS_SEG_MANO")
ids_existentes = {fila[0] for fila in cursor.fetchall()}

# Parsear página inicial
soup = bs(driver.page_source, "lxml")
lista_id_vehiculos = [div["data-ad-id"] for div in soup.select("div[data-ad-id]")]

for i in range(1, len(lista_id_vehiculos) + 1):
    id_vehiculo = lista_id_vehiculos[i - 1]

    if id_vehiculo not in ids_existentes:
        try:
            print(f"Procesando vehículo {i}/{len(lista_id_vehiculos)} - ID: {id_vehiculo}")

            # XPath dinámico basado en el índice
            xpath_anuncio = f"/html/body/div[2]/div/div[2]/div[1]/div[1]/main/div[4]/section/div[1]/div[{i}]/div/div"
            anuncio_vehiculo = driver.find_element(By.XPATH, xpath_anuncio)
            anuncio_vehiculo.click()

            # Esperamos a que cargue la nueva página
            time.sleep(random.uniform(8, 15))

            url_vehiculo = driver.current_url
            soup_vehiculo = bs(driver.page_source, "lxml")

            # Título
            title_element = soup_vehiculo.select_one('h1.mt-TitleBasic-title')
            if not title_element:
                raise Exception("No se encontró el título del vehículo")

            palabras = title_element.get_text(strip=True).split()
            marca_vehiculo = palabras[0]
            modelo_vehiculo = " ".join(palabras[1:])

            # Precio
            precio = None
            string_precio = soup_vehiculo.find("p", class_="mt-CardAdPrice-cashAmount")
            if string_precio:
                texto_precio = string_precio.text.strip().replace('.', '').replace('€', '').strip()
                precio = float(texto_precio)

            # Datos principales
            tabla_datos_vehiculo = soup_vehiculo.select('ul.mt-PanelAdDetails-data li.mt-PanelAdDetails-dataItem')
            dicc = extraer_datos_tecnicos(tabla_datos_vehiculo)

            # Fecha publicación
            hoy = datetime.now()
            fecha_actual = hoy.strftime("%d/%m")
            string_fecha_publicacion = soup_vehiculo.find("p", class_="mt-PanelAdInfo-published")
            fecha_final_publicacion = None
            anyomes_publicacion = None

            if string_fecha_publicacion:
                texto = string_fecha_publicacion.text.strip()
                fecha_str = texto.split(",")[0].replace("Publicado: ", "").strip()[0:5]
                fecha_publicacion_dt = datetime.strptime(fecha_str, "%d/%m")
                fecha_actual_dt = datetime.strptime(fecha_actual, "%d/%m")

                anyo = hoy.year - 1 if fecha_publicacion_dt > fecha_actual_dt else hoy.year
                fecha_final_publicacion = datetime.strptime(f"{fecha_str}/{anyo}", "%d/%m/%Y").date()
                anyomes_publicacion = int(f"{anyo}{fecha_publicacion_dt.month:02d}")

            # Insertar en BBDD
            consulta = """
                INSERT OR IGNORE INTO TX_VEHICULOS_SEG_MANO (
                    pk_anuncio_id, marca, modelo, precio, combustible, anyo_vehiculo,
                    kilometraje, potencia, num_puertas, num_plazas, tipo_cambio,
                    tipo_vehiculo, cilindrada_motor, color, provincia, etiqueta_eco,
                    origen_anuncio, fecha_publicacion, anyomes_publicacion, url
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """

            valores = (
                id_vehiculo,
                marca_vehiculo,
                modelo_vehiculo,
                precio,
                dicc.get("combustible"),
                int(dicc.get("anyo")) if dicc.get("anyo") else None,
                int(dicc.get("kms")) if dicc.get("kms") else None,
                int(dicc.get("potencia")) if dicc.get("potencia") else None,
                int(dicc.get("puertas")) if dicc.get("puertas") else None,
                int(dicc.get("plazas")) if dicc.get("plazas") else None,
                dicc.get("cambio"),
                dicc.get("carroceria"),
                int(dicc.get("cilindrada")) if dicc.get("cilindrada") else None,
                dicc.get("color"),
                dicc.get("ciudad"),
                dicc.get("etiqueta"),
                "WebScraping",
                fecha_final_publicacion,
                anyomes_publicacion,
                url_vehiculo
            )

            cursor.execute(consulta, valores)
            con.commit()

            time.sleep(random.uniform(15, 15))

            driver.back()
            time.sleep(random.uniform(10, 10))

        except Exception as e:
            print(f"❌ Error al procesar ID {id_vehiculo}: {e}")
            try:
                driver.back()
                time.sleep(3)
            except:
                print("⚠️ Fallo al volver atrás, recargando página...")
                driver.get("https://www.coches.net/segunda-mano/?MakeIds%5B0%5D=7&ModelIds%5B0%5D=0")
                time.sleep(random.uniform(3, 5))


Procesando vehículo 1/65 - ID: 59973168
Procesando vehículo 2/65 - ID: 59371905
Procesando vehículo 3/65 - ID: 59590083
Procesando vehículo 4/65 - ID: 60227457
Procesando vehículo 5/65 - ID: 60067512
Procesando vehículo 6/65 - ID: 60199438
Procesando vehículo 7/65 - ID: 60199438
Procesando vehículo 8/65 - ID: 57049267
Procesando vehículo 9/65 - ID: 57049267
❌ Error al procesar ID 57049267: Message: element not interactable
  (Session info: chrome=136.0.7103.93)
Stacktrace:
	GetHandleVerifier [0x0041FC53+61635]
	GetHandleVerifier [0x0041FC94+61700]
	(No symbol) [0x0024044E]
	(No symbol) [0x00289B4B]
	(No symbol) [0x0027E5A5]
	(No symbol) [0x002AD29C]
	(No symbol) [0x0027E034]
	(No symbol) [0x002AD514]
	(No symbol) [0x002CE61B]
	(No symbol) [0x002AD096]
	(No symbol) [0x0027C840]
	(No symbol) [0x0027D6A4]
	GetHandleVerifier [0x006A4573+2701795]
	GetHandleVerifier [0x0069FCF6+2683238]
	GetHandleVerifier [0x006BAA3E+2793134]
	GetHandleVerifier [0x00436915+155013]
	GetHandleVerifier [0x0043C

Miramos cuántos coches se han cargado en la BBDD:

In [12]:
import pandas as pd
import sqlite3

# Conexión a la BBDD:
con = sqlite3.connect("./include/db_vehiculos.db")

# Vamos a ver qué nos devuelve un select sobre la tabla:
query = """
SELECT * FROM TX_VEHICULOS_SEG_MANO;
"""

# Leemos los resultados en un DataFrame de pandas
df = pd.read_sql_query(query, con)

# Mostramos los primeros registros (aún no hay):
df

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,...,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,56781291,BMW,Serie 3 325xi E90,8290.0,Gasolina,2006,179000,218,4,5,...,Berlina,2497.0,Negro,Barcelona,None,WebScraping,2025-03-25,202503,2025-05-11 17:56:58,https://www.coches.net/bmw-serie-3-325xi-e90-4...
1,57049267,BMW,Serie 1 118d,12000.0,Diesel,2014,200000,143,3,5,...,Berlina,1995.0,Blanco,Murcia,B (amarilla),WebScraping,2025-05-04,202505,2025-05-11 17:50:22,https://www.coches.net/bmw-serie-1-118d-3p-die...
2,57735226,BMW,Serie 1 120d,32900.0,Diesel,2024,25900,190,5,5,...,Berlina,1995.0,Negro,Madrid,C (verde),WebScraping,2025-04-29,202504,2025-05-11 18:04:08,https://www.coches.net/bmw-serie-1-120d-5p-die...
3,59030836,BMW,Serie 3 318d Touring,18890.0,Diesel,2019,114331,150,5,5,...,Familiar,1995.0,Negro,Madrid,C (verde),WebScraping,2025-04-04,202504,2025-05-11 17:37:10,https://www.coches.net/bmw-serie-3-318d-tourin...
4,59371905,BMW,X2 sDrive20i DCT,26900.0,Gasolina,2018,90000,192,5,5,...,SUV,1998.0,Azul,Barcelona,None,WebScraping,2025-05-04,202505,2025-05-11 17:46:43,https://www.coches.net/bmw-x2-sdrive20i-dct-5p...
5,59590083,BMW,Serie 3 320d Auto.,39900.0,Híbrido,2024,22907,190,4,5,...,Berlina,1995.0,Blanco,Toledo,ECO (azul/verde),WebScraping,2025-02-03,202502,2025-05-11 17:47:20,https://www.coches.net/bmw-serie-3-320d-auto.-...
6,59973168,BMW,Serie 1 116d,9990.0,Diesel,2013,142133,116,5,5,...,Berlina,1995.0,Azul,Cantabria,B (amarilla),WebScraping,2025-02-27,202502,2025-05-11 17:46:05,https://www.coches.net/bmw-serie-1-116d-5p-die...
7,60067512,BMW,X3 XDRIVE20D,9500.0,Diesel,2008,217000,177,5,5,...,SUV,1995.0,Blanco,Madrid,B (amarilla),WebScraping,2025-01-15,202501,2025-05-11 17:48:35,https://www.coches.net/bmw-x3-xdrive20d-5p-die...
8,60188159,BMW,Serie 3 320d,17500.0,Diesel,2013,170000,184,4,5,...,Berlina,1995.0,Negro,Murcia,B (amarilla),WebScraping,2025-05-02,202505,2025-05-11 17:59:42,https://www.coches.net/bmw-serie-3-320d-effici...
9,60199438,BMW,Serie 1 116d,14990.0,Diesel,2019,130000,116,5,5,...,Berlina,1496.0,Gris / Plata,Sevilla,C (verde),WebScraping,2025-03-27,202503,2025-05-11 17:49:11,https://www.coches.net/bmw-serie-1-116d-5p-die...


Se han cargado 24 coches. Ha intentado scrapear 65 pero en los que hay diferencias en la estructura de algún campo no se han podido cargar.

Vamos a implementar que recorra más de 1 página para que busque más de 65 registros.

In [29]:
import time
from datetime import datetime, timedelta
import random
import sqlite3
import os
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup as bs
from include.utils.funciones import *

# Conexión a la BBDD:
con = sqlite3.connect("./include/db_vehiculos.db")
cursor = con.cursor()

# User agents aleatorios:
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:99.0) Gecko/20100101 Firefox/99.0",
]

options = uc.ChromeOptions()
selected_ua = random.choice(USER_AGENTS)
options.add_argument(f"user-agent={selected_ua}")
options.add_argument("--no-sandbox")
options.add_argument("--disable-blink-features=AutomationControlled")
# options.add_argument("--headless")  # opcional

# Inicializar navegador
driver = uc.Chrome(options=options)


# Marcas de coche scrapeadas:
lista_marcas = ['BMW', 'VOLKSWAGEN', 'MERCEDES-BENZ', 'AUDI', 'PEUGEOT', 
                'FORD', 'RENAULT', 'OPEL', 'CITROEN', 'SEAT']



pagina = 1
while pagina < 5:

    # Cambiar según la marca o marcas que queramos guardar:

    url_todas_marcas = f'https://www.coches.net/segunda-mano/?MakeIds%5B0%5D=4&MakeIds%5B1%5D=7&MakeIds%5B2%5D=11&MakeIds%5B3%5D=15&MakeIds%5B4%5D=28&MakeIds%5B5%5D=32&MakeIds%5B6%5D=33&MakeIds%5B7%5D=35&MakeIds%5B8%5D=39&MakeIds%5B9%5D=47&ModelIds%5B0%5D=0&ModelIds%5B1%5D=0&ModelIds%5B2%5D=0&ModelIds%5B3%5D=0&ModelIds%5B4%5D=0&ModelIds%5B5%5D=0&ModelIds%5B6%5D=0&ModelIds%5B7%5D=0&ModelIds%5B8%5D=0&ModelIds%5B9%5D=0&Versions%5B0%5D=&Versions%5B1%5D=&Versions%5B2%5D=&Versions%5B3%5D=&Versions%5B4%5D=&Versions%5B5%5D=&Versions%5B6%5D=&Versions%5B7%5D=&Versions%5B8%5D=&Versions%5B9%5D=&pg={pagina}'
    # url_bmw = f"https://www.coches.net/segunda-mano/?MakeIds%5B0%5D=7&ModelIds%5B0%5D=0&pg={pagina}"

    url_pagina = url_todas_marcas
    driver.get(url_pagina)
    time.sleep(random.uniform(4, 6))

    # Aceptar cookies si aparecen
    if pagina == 1:
        try:
            Cookies = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div/div/div[3]/button[3]")
            Cookies.click()
            time.sleep(1)
        except:
            pass

    # Lista de IDs en la BBDD
    cursor.execute("SELECT PK_ANUNCIO_ID FROM TX_VEHICULOS_SEG_MANO")
    ids_existentes = {fila[0] for fila in cursor.fetchall()}

    soup = bs(driver.page_source, "lxml")
    lista_id_vehiculos = [div["data-ad-id"] for div in soup.select("div[data-ad-id]")]

    if not lista_id_vehiculos:
        print("✅ No hay más anuncios en esta página. Fin del scraping.")
        break

    for i in range(1, len(lista_id_vehiculos) + 1):
        id_vehiculo = lista_id_vehiculos[i - 1]

        if id_vehiculo not in ids_existentes:
            try:
                print(f"Procesando vehículo {i}/{len(lista_id_vehiculos)} - ID: {id_vehiculo}")

                xpath_anuncio = f"/html/body/div[2]/div/div[2]/div[1]/div[1]/main/div[4]/section/div[1]/div[{i}]/div/div"
                anuncio_vehiculo = driver.find_element(By.XPATH, xpath_anuncio)
                anuncio_vehiculo.click()

                time.sleep(random.uniform(8, 15))

                scroll_y = random.randint(200, 1000)
                driver.execute_script(f"window.scrollBy(0, {scroll_y});")
                time.sleep(random.uniform(1, 2))
                driver.execute_script(f"window.scrollBy(0, {-scroll_y // 2});")
                time.sleep(random.uniform(1, 2))

                try:
                    actions = ActionChains(driver)
                    body = driver.find_element(By.TAG_NAME, "body")
                    actions.move_to_element_with_offset(body, random.randint(0, 300), random.randint(0, 300)).perform()
                    time.sleep(random.uniform(8, 15))
                except Exception as e:
                    print(f"⚠️ No se pudo simular movimiento del ratón: {e}")

                url_vehiculo = driver.current_url
                soup_vehiculo = bs(driver.page_source, "lxml")

                title_element = soup_vehiculo.select_one('h1.mt-TitleBasic-title')
                if not title_element:
                    raise Exception("No se encontró el título del vehículo")

                palabras = title_element.get_text(strip=True).split()
                marca_vehiculo = palabras[0]
                modelo_vehiculo = " ".join(palabras[1:])

                precio = None
                string_precio = soup_vehiculo.find("p", class_="mt-CardAdPrice-cashAmount")
                if string_precio:
                    texto_precio = string_precio.text.strip().replace('.', '').replace('€', '').strip()
                    precio = float(texto_precio)

                tabla_datos_vehiculo = soup_vehiculo.select('ul.mt-PanelAdDetails-data li.mt-PanelAdDetails-dataItem')
                dicc = extraer_datos_tecnicos(tabla_datos_vehiculo)

                hoy = datetime.now()
                fecha_actual = hoy.strftime("%d/%m")
                string_fecha_publicacion = soup_vehiculo.find("p", class_="mt-PanelAdInfo-published")
                fecha_final_publicacion = None
                anyomes_publicacion = None

                # if string_fecha_publicacion:
                #     texto = string_fecha_publicacion.text.strip()
                #     fecha_str = texto.split(",")[0].replace("Publicado: ", "").strip()[0:5]
                #     fecha_publicacion_dt = datetime.strptime(fecha_str, "%d/%m")
                #     fecha_actual_dt = datetime.strptime(fecha_actual, "%d/%m")
                #     anyo = hoy.year - 1 if fecha_publicacion_dt > fecha_actual_dt else hoy.year
                #     fecha_final_publicacion = datetime.strptime(f"{fecha_str}/{anyo}", "%d/%m/%Y").date()
                #     anyomes_publicacion = int(f"{anyo}{fecha_publicacion_dt.month:02d}")


                if string_fecha_publicacion:
                    texto = string_fecha_publicacion.text.strip().lower()

                    if "hace" in texto or "hoy" in texto:
                        fecha_final_publicacion = datetime.now().date()
                    elif "ayer" in texto:
                        fecha_final_publicacion = (datetime.now() - timedelta(days=1)).date()
                    else:
                        try:
                            # Ejemplo: "Publicado: 12/03, hace 3 días"
                            fecha_str = texto.split(",")[0].replace("publicado: ", "").strip()[0:5]
                            fecha_publicacion_dt = datetime.strptime(fecha_str, "%d/%m")
                            fecha_actual_dt = datetime.strptime(fecha_actual, "%d/%m")

                            # Ajuste de año si es diciembre vs enero
                            anyo = hoy.year - 1 if fecha_publicacion_dt > fecha_actual_dt else hoy.year
                            fecha_final_publicacion = datetime.strptime(f"{fecha_str}/{anyo}", "%d/%m/%Y").date()
                        except Exception as e:
                            print(f"⚠️ No se pudo interpretar la fecha: {texto} - {e}")
                            fecha_final_publicacion = None

                    if fecha_final_publicacion:
                        anyomes_publicacion = int(f"{fecha_final_publicacion.year}{fecha_final_publicacion.month:02d}")
                    else:
                        anyomes_publicacion = None


                consulta = """
                    INSERT OR IGNORE INTO TX_VEHICULOS_SEG_MANO (
                        pk_anuncio_id, marca, modelo, precio, combustible, anyo_vehiculo,
                        kilometraje, potencia, num_puertas, num_plazas, tipo_cambio,
                        tipo_vehiculo, cilindrada_motor, color, provincia, etiqueta_eco,
                        origen_anuncio, fecha_publicacion, anyomes_publicacion, url
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """

                valores = (
                    id_vehiculo,
                    marca_vehiculo,
                    modelo_vehiculo,
                    precio,
                    dicc.get("combustible"),
                    int(dicc.get("anyo")) if dicc.get("anyo") else None,
                    int(dicc.get("kms")) if dicc.get("kms") else None,
                    int(dicc.get("potencia")) if dicc.get("potencia") else None,
                    int(dicc.get("puertas")) if dicc.get("puertas") else None,
                    int(dicc.get("plazas")) if dicc.get("plazas") else None,
                    dicc.get("cambio"),
                    dicc.get("carroceria"),
                    int(dicc.get("cilindrada")) if dicc.get("cilindrada") else None,
                    dicc.get("color"),
                    dicc.get("ciudad"),
                    dicc.get("etiqueta"),
                    "WebScraping",
                    fecha_final_publicacion,
                    anyomes_publicacion,
                    url_vehiculo
                )

                cursor.execute(consulta, valores)
                con.commit()

                time.sleep(random.uniform(15, 15))
                driver.back()
                time.sleep(random.uniform(10, 10))

                if (i % 5 == 0) or (i % 8 == 0):
                    print("⏸️ Pausa larga simulando descanso del usuario...")
                    time.sleep(random.uniform(30, 30))

            except Exception as e:
                print(f"❌ Error al procesar ID {id_vehiculo}: {e}")
                try:
                    driver.back()
                    time.sleep(3)
                except:
                    pass

    pagina += 1

# Cierre final
driver.quit()
con.close()


Procesando vehículo 1/65 - ID: 57455675
Procesando vehículo 2/65 - ID: 59114694
Procesando vehículo 3/65 - ID: 60486796
Procesando vehículo 4/65 - ID: 60357138
Procesando vehículo 5/65 - ID: 60425898
❌ Error al procesar ID 60425898: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div/div[2]/div[1]/div[1]/main/div[4]/section/div[1]/div[5]/div/div"}
  (Session info: chrome=136.0.7103.93); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x004EFC53+61635]
	GetHandleVerifier [0x004EFC94+61700]
	(No symbol) [0x003105D3]
	(No symbol) [0x0035899E]
	(No symbol) [0x00358D3B]
	(No symbol) [0x003A0E12]
	(No symbol) [0x0037D2E4]
	(No symbol) [0x0039E61B]
	(No symbol) [0x0037D096]
	(No symbol) [0x0034C840]
	(No symbol) [0x0034D6A4]
	GetHandleVerifier [0x00774573+2701795]
	GetHandleVerifier [0x0076FCF6+2683238]
	GetHandleV

KeyboardInterrupt: 

In [30]:
import pandas as pd
import sqlite3

# Conexión a la BBDD:
con = sqlite3.connect("./include/db_vehiculos.db")

# Vamos a ver qué nos devuelve un select sobre la tabla:
query = """
SELECT * FROM TX_VEHICULOS_SEG_MANO
--where
--fecha_carga > '2025-05-11 22:33:00'
order by
    fecha_carga desc
;
"""

# Leemos los resultados en un DataFrame de pandas
df = pd.read_sql_query(query, con)

# Mostramos los primeros registros (aún no hay):
df

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,...,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,60078270,RENAULT,Mégane Limited Blue dCi,12990.0,Diesel,2020.0,155000.0,115.0,5.0,5.0,...,Berlina,1461.0,Blanco,Sevilla,C (verde),WebScraping,2025-05-13,202505.0,2025-05-13 21:41:38,https://www.coches.net/renault-megane-limited-...
1,60533831,PEUGEOT,308 Style 1.5 BlueHDi,12950.0,Diesel,2019.0,119000.0,130.0,5.0,5.0,...,Berlina,1499.0,Blanco,Madrid,C (verde),WebScraping,2025-05-13,202505.0,2025-05-13 21:39:56,https://www.coches.net/peugeot-308-5p-style-1....
2,60550466,MERCEDES-BENZ,Clase C C 220 CDI Avantgarde,8100.0,Diesel,2008.0,260000.0,170.0,4.0,5.0,...,Berlina,2148.0,Azul,Murcia,B (amarilla),WebScraping,2025-05-13,202505.0,2025-05-13 21:31:25,https://www.coches.net/mercedes-benz-clase-c-c...
3,60549892,FORD,Fiesta 1.1 PFI GLP Trend,9990.0,None,2020.0,106000.0,75.0,5.0,5.0,...,Berlina,1084.0,Blanco,Málaga,ECO (azul/verde),WebScraping,2025-05-13,202505.0,2025-05-13 21:29:42,https://www.coches.net/ford-fiesta-11-pfi-glp-...
4,60550460,RENAULT,Mégane Dynamique dCi 110 eco2 E5,6290.0,Diesel,2010.0,193000.0,110.0,5.0,5.0,...,Berlina,1461.0,None,Madrid,B (amarilla),WebScraping,2025-05-13,202505.0,2025-05-13 21:28:23,https://www.coches.net/renault-megane-dynamiqu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,59973168,BMW,Serie 1 116d,9990.0,Diesel,2013.0,142133.0,116.0,5.0,5.0,...,Berlina,1995.0,Azul,Cantabria,B (amarilla),WebScraping,2025-02-27,202502.0,2025-05-11 17:46:05,https://www.coches.net/bmw-serie-1-116d-5p-die...
254,60439306,BMW,Serie 3 M3 Competition,84900.0,Gasolina,2023.0,26000.0,510.0,4.0,5.0,...,Berlina,2993.0,Gris / Plata,Pontevedra,None,WebScraping,2025-03-05,202503.0,2025-05-11 17:37:18,https://www.coches.net/bmw-serie-3-m3-competit...
255,59030836,BMW,Serie 3 318d Touring,18890.0,Diesel,2019.0,114331.0,150.0,5.0,5.0,...,Familiar,1995.0,Negro,Madrid,C (verde),WebScraping,2025-04-04,202504.0,2025-05-11 17:37:10,https://www.coches.net/bmw-serie-3-318d-tourin...
256,60330276,BMW,Serie 3 330e xDrive Touring,34990.0,Híbrido Enchufable,2020.0,66011.0,292.0,5.0,5.0,...,Familiar,1998.0,Azul,Barcelona,None,WebScraping,2025-04-12,202504.0,2025-05-11 17:37:02,https://www.coches.net/bmw-serie-3-330e-xdrive...


In [31]:
df.groupby("marca")["pk_anuncio_id"].count().sort_values(ascending=False)

marca
BMW              83
RENAULT          32
SEAT             30
VOLKSWAGEN       23
FORD             20
CITROEN          17
OPEL             16
PEUGEOT          15
MERCEDES-BENZ    11
AUDI              7
176.885           1
176.910           1
177.052           1
177.946           1
Name: pk_anuncio_id, dtype: int64

In [32]:
import pandas as pd
import sqlite3

# Conexión a la BBDD:
con = sqlite3.connect("./include/db_vehiculos.db")

# Vamos a ver qué nos devuelve un select sobre la tabla:
query = """
SELECT * FROM TX_VEHICULOS_SEG_MANO
where
--fecha_carga > '2025-05-13 19:33:00'
marca not in ('BMW', 'VOLKSWAGEN', 'MERCEDES-BENZ', 'AUDI', 'PEUGEOT', 
                'FORD', 'RENAULT', 'OPEL', 'CITROEN', 'SEAT')
order by
    fecha_carga desc
;
"""

# Leemos los resultados en un DataFrame de pandas
df = pd.read_sql_query(query, con)

# # Mostramos los primeros registros (aún no hay):
df

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,...,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,60543518,176.885,AUDI de segunda mano y ocasión,17990.0,None,None,None,None,None,None,...,None,None,None,None,None,WebScraping,None,None,2025-05-13 19:52:59,https://www.coches.net/segunda-mano/?MakeIds%5...
1,60543182,176.910,AUDI de segunda mano y ocasión,15990.0,None,None,None,None,None,None,...,None,None,None,None,None,WebScraping,None,None,2025-05-13 19:21:08,https://www.coches.net/segunda-mano/?MakeIds%5...
2,60549062,177.052,AUDI de segunda mano y ocasión,7970.0,None,None,None,None,None,None,...,None,None,None,None,None,WebScraping,None,None,2025-05-13 18:45:16,https://www.coches.net/segunda-mano/?MakeIds%5...
3,60445302,177.946,AUDI de segunda mano y ocasión,39900.0,None,None,None,None,None,None,...,None,None,None,None,None,WebScraping,None,None,2025-05-12 19:11:52,https://www.coches.net/segunda-mano/?MakeIds%5...
